# Numerical optimization

Using simple examples, this tutorial shows how to do an optimization with optimagic. More details on the topics covered here can be found in the [how to guides](../how_to/index.md).

In [ ]:
import numpy as np
import optimagic as om
import pandas as pd

## Basic usage of `minimize`

In [ ]:
def sphere(params):
    return params @ params

In [ ]:
res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
)

res.params.round(5)

## `params` do not have to be vectors

In optimagic, params can by arbitrary [pytrees](https://jax.readthedocs.io/en/latest/pytrees.html). Examples are (nested) dictionaries of numbers, arrays and pandas objects. 

In [ ]:
def dict_sphere(params):
    return params["a"] ** 2 + params["b"] ** 2 + (params["c"] ** 2).sum()

In [ ]:
res = om.minimize(
    fun=dict_sphere,
    params={"a": 0, "b": 1, "c": pd.Series([2, 3, 4])},
    algorithm="scipy_powell",
)

res.params

## The result contains all you need to know

In [ ]:
res = om.minimize(
    fun=dict_sphere,
    params={"a": 0, "b": 1, "c": pd.Series([2, 3, 4])},
    algorithm="scipy_neldermead",
)
res

## You can visualize the convergence

In [ ]:
fig = om.criterion_plot(res, max_evaluations=300)
fig.show(renderer="png")

In [ ]:
fig = om.params_plot(
    res,
    max_evaluations=300,
    # optionally select a subset of parameters to plot
    selector=lambda params: params["c"],
)
fig.show(renderer="png")

## There are many optimizers

If you install some optional dependencies, you can choose from a large (and growing) set of optimization algorithms -- all with the same interface!

For example, we wrap optimizers from `scipy.optimize`, `nlopt`, `cyipopt`, `pygmo`, `fides`, `tao` and others. 

We also have some optimizers that are not part of other packages. Examples are a `parallel Nelder-Mead` algorithm, The `BHHH` algorithm and a `parallel Pounders` algorithm.

See the full list [here](../how_to_guides/optimization/how_to_specify_algorithm_and_algo_options

## You can add bounds

In [ ]:
bounds = om.Bounds(lower=np.arange(5) - 2, upper=np.array([10, 10, 10, np.inf, np.inf]))

res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    bounds=bounds,
)

res.params.round(5)

## You can fix parameters 

In [ ]:
res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    constraints=[{"loc": [1, 3], "type": "fixed"}],
)

res.params.round(5)

## Or impose other constraints

As an example, let's impose the constraint that the first three parameters are valid probabilities, i.e. they are between zero and one and sum to one:

In [ ]:
res = om.minimize(
    fun=sphere,
    params=np.array([0.1, 0.5, 0.4, 4, 5]),
    algorithm="scipy_lbfgsb",
    constraints=[{"loc": [0, 1, 2], "type": "probability"}],
)

res.params.round(5)

For a full overview of the constraints we support and the corresponding syntaxes, check out [the documentation](../how_to/how_to_constraints.md).

Note that `"scipy_lbfgsb"` is not a constrained optimizer. If you want to know how we achieve this, check out [the explanations](../explanation/implementation_of_constraints.md).

## There is also maximize

If you ever forgot to switch back the sign of your criterion function after doing a maximization with `scipy.optimize.minimize`, there is good news:

In [ ]:
def upside_down_sphere(params):
    return -params @ params

In [ ]:
res = om.maximize(
    fun=upside_down_sphere,
    params=np.arange(5),
    algorithm="scipy_bfgs",
)

res.params.round(5)

optimagic got your back.

## You can provide closed form derivatives

In [ ]:
def sphere_gradient(params):
    return 2 * params

In [ ]:
res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    jac=sphere_gradient,
)
res.params.round(5)

## Or use parallelized numerical derivatives

In [ ]:
res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    numdiff_options=om.NumdiffOptions(n_cores=6),
)

res.params.round(5)

## Turn local optimizers global with multistart

Multistart optimization requires finite soft bounds on all parameters. Those bounds will
be used for sampling but not enforced during optimization.

In [ ]:
bounds = om.Bounds(soft_lower=np.full(10, -5), soft_upper=np.full(10, 15))

res = om.minimize(
    fun=sphere,
    params=np.arange(10),
    algorithm="scipy_neldermead",
    bounds=bounds,
    multistart=om.MultistartOptions(convergence_max_discoveries=5),
)
res.params.round(5)

## And plot the criterion history of all local optimizations

In [ ]:
fig = om.criterion_plot(res)
fig.show(renderer="png")

## Exploit the structure of your optimization problem

Many estimation problems have a least-squares structure. If so, specialized optimizers that exploit this structure can be much faster than standard optimizers. The `sphere` function from above is the simplest possible least-squarse problem you could imagine: the least-squares residuals are just the params. 

To use least-squares optimizers in optimagic, you need to declare mark your function with 
a decorator and return the least-squares residuals instead of the aggregated function value. 

More details can be found [here](../how_to/how_to_criterion_function.md)

In [ ]:
@om.mark.least_squares
def ls_sphere(params):
    return params

In [ ]:
res = om.minimize(
    fun=ls_sphere,
    params=np.arange(5),
    algorithm="pounders",
)
res.params.round(5)

Of course, any least-squares problem can also be solved with a standard optimizer. 

There are also specialized optimizers for likelihood functions. 

## Using and reading persistent logging

For long-running and difficult optimizations, it can be worthwhile to store the progress in a persistent log file. You can do this by providing a path to the `logging` argument:

In [ ]:
res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    logging="my_log.db",
    log_options={"if_database_exists": "replace"},
)

You can read the entries in the log file (while the optimization is still running or after it has finished) as follows:

In [ ]:
reader = om.OptimizeLogReader("my_log.db")
reader.read_history().keys()

For more information on what you can do with the log file and LogReader object, check out [the logging tutorial](../how_to/how_to_logging.ipynb)

The persistent log file is always instantly synchronized when the optimizer tries a new parameter vector. This is very handy if an optimization has to be aborted and you want to extract the current status. It can be displayed in `criterion_plot` and `params_plot`, even while the optimization is running. 

## Customize your optimizer

Most algorithms have a few optional arguments. Examples are convergence criteria or tuning parameters. You can find an overview of supported arguments [here](../how_to/how_to_specify_algorithm_and_algo_options.md).

In [ ]:
algo_options = {
    "convergence.ftol_rel": 1e-9,
    "stopping.maxiter": 100_000,
}

res = om.minimize(
    fun=sphere,
    params=np.arange(5),
    algorithm="scipy_lbfgsb",
    algo_options=algo_options,
)
res.params.round(5)